## MILP tutorial 1 : make a linear model

This tutorial is the first of a more elementary serie aimed to explain the concepts behind a MILP model and how to set up one. This tutorial serie is applied to energy system optimisations, but can be extended to any system optimisation.

A Mixed Integer Linear Programming problem consists in optimizing a set of variables (continuous or discrete) in order to minimize a cost function. Equality or inequality constraints can be set between the variables to make the model more complex.

An example of linear problem would be:

-   with x a continuous variable
-   minimize(x)
-   x >= 3

x = 3 is the obvious answer to this problem

A similarly simple MILP problem would be:

-   with x a discrete variable
-   minimize(x)
-   x >= 2.5

x = 3 is again the obvious answer to the problem

###   Imports

Many python module allow to intuitively set-up a MILP model and does the interface with the solver, including mip, pyomo, ortools... We will in this tutorial use the docplex module that has a limited free version sufficient to run these tutorials. The online documentation can be found here:

https://www.ibm.com/docs/en/icos/22.1.1?topic=docplex-python-modeling-api

In [9]:
# Import the Docplex Model object that manages the problem and its resolution
from docplex.mp.model import Model

###   Create a linear model

In this section we create a minimalistic energy mix problem where we have two production means that can be used in order to answer to a energy load. The problem properties are the following:

-   Load : 20 MW
-   Prod1 :
    - maximum power : 15MW
    - price : 1€ / MW
-   Prod2 :
    - maximum power : 15MW
    - price : 5€ / MW

Intuitively, the Prod1 mean is expected to be fully used, and the Prod2 is expected to be used to supply the remaining load.

To create this model and solve it, a Model object needs to be created. It will 
- provide the variables;
- store the constraints;
- store the optimisation function;
- solve the problem.

In [10]:
model = Model()

The variable can now be accessed. Our problem has two variables: one per production mean.

Note that:
- The production being positive, a lower bound is set.
- The production of both means being lower than 15MW, this value is set as upper bound.

These lower and upper bound will in the end create one inequality constraint each per variable.

In [11]:
production_1 = model.continuous_var(name="production_1", lb=0, ub=15)
production_2 = model.continuous_var(name="production_2", lb=0, ub=15)

The constraints can now be added. In our model, we need the sum of these productions to equal the load (20).

In [12]:
model += production_1 + production_2 == 20

We can now provide the objective of the model that we are willing to minimize.

In [13]:
model.set_objective("min", 1 * production_1 + 5 * production_2)

For education purposes, it can be very informative to write down the model that the solver will read, it can be done using the command:

In [14]:
model.export_as_lp("my_lp_problem.lp")

'my_lp_problem.lp'

The problem can now be solved and the results imported from the solver. We find the couple (15, 5) as we expected.

In [15]:
model.solve(log_output = True)

print(f"\nResults: {production_1.solution_value}, {production_2.solution_value}")

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 1 rows and 2 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)

Results: 15.0, 5.0
